In [2]:
nltk.download()
#https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
import nltk.tokenize as nt
import nltk

#reference_tasks = ['create','hold','release','schedule','release','invoice','ship']
#reference_entities = ['order','node','organization','line','address','shipment']
filenames = ['create_words.out','hold_words.out','inventory_words.out','invoice_words.out',
             'release_words.out','schedule_words.out','ship_words.out']

class nlpdict(dict): 
  
    # __init__ function 
    def __init__(self): 
        self = dict() 
          
    # Function to add key:value 
    def add(self, key, value): 
        self[key] = value

nlpdictobj = nlpdict()
for fname in filenames:
    with open(fname, "r") as fin:
        mylist = fin.read().splitlines()
        pos=nltk.pos_tag(mylist)
        nlpdictobj.add(fname,pos)

print(nlpdictobj)

{'create_words.out': [('added', 'VBD'), ('available', 'JJ'), ('consumed', 'VBN'), ('created', 'VBN'), ('creation', 'NN'), ('current', 'JJ'), ('default', 'NN'), ('demand', 'NN'), ('exchange', 'NN'), ('indicates', 'VBZ'), ('item', 'JJ'), ('line', 'NN'), ('made', 'VBD'), ('need', 'MD'), ('new', 'JJ'), ('order', 'NN'), ('payment', 'NN'), ('purpose', 'JJ'), ('return', 'NN'), ('set', 'VBN'), ('status', 'NN'), ('system', 'NN'), ('used', 'VBN'), ('valid', 'JJ'), ('value', 'NN'), ('whether', 'IN'), ('work', 'NN')], 'hold_words.out': [('applied', 'VBN'), ('customer', 'NN'), ('default', 'NN'), ('hold', 'VBP'), ('limit', 'NN'), ('line', 'NN'), ('order', 'NN'), ('payment', 'NN'), ('placed', 'VBD'), ('resolved', 'JJ'), ('transaction', 'NN'), ('type', 'NN'), ('user', 'NN'), ('valid', 'JJ'), ('value', 'NN'), ('whether', 'IN')], 'inventory_words.out': [('adjustment', 'NN'), ('allow', 'NN'), ('always', 'RB'), ('applies', 'VBZ'), ('arrived', 'VBN'), ('associated', 'VBN'), ('attribute', 'JJ'), ('audit', '

In [6]:
class wordcombo:
    concept = ''
    nouns,verbs,adjectives=set(),set(),set()
    def __init__(self,c,n,v,a):
        self.concept = c
        self.nouns = n
        self.verbs = v
        self.adjectives = a

wordcombos=[]
for key in nlpdictobj:
    a,c,n,v=set(),set(),set(),set()
    for word,tag in nlpdictobj[key]:
        if tag.startswith("N"):
            n.add(word)
        elif tag.startswith("V"):
            v.add(word)
        elif tag.startswith("J"):
            a.add(word)
    mywc = wordcombo(key,n,v,a)
    wordcombos.append(mywc)

print(len(wordcombos))


7


In [23]:
#print(wordcombos[0].nouns)
rulesetfiles = ['create_rules.out','hold_rules.out','inventory_rules.out','invoice_rules.out',
             'release_rules.out','schedule_rules.out','ship_rules.out']
for rulesetfile in rulesetfiles:
    with open(rulesetfile, 'r') as frin:
        wordfile=rulesetfile.split('_')[0]
        shortrulefile = wordfile + "_shortrules.out"
        wordfile+= '_words.out'
        with open(shortrulefile, 'w') as frout:
            mywc = None
            for wc in wordcombos:
                if wc.concept == wordfile:
                    mywc = wc
                    break
            if mywc == None:
                print("Skipping " + rulesetfile)
                continue
            nounRules, verbRules = set(),set()
            mylist = frin.read().splitlines()
            for row in mylist:
                #check if antecedent has {noun, verb} or {noun, noun}
                #print(row)
                words = row.split(',')
                #There are 7 parts to a row.
                #print(words)
                ante,cons,lift,conv=set(words[1].split(' ')),set(words[2].split(' ')),words[5],words[6]
                antelist.append(words[1]), conslist.append(words[2])
                #print('checking ' + str(ante) + ' with ' + str(mywc.nouns))
                antenouns,anteverbs = mywc.nouns.intersection(ante),mywc.verbs.intersection(ante)
                consnouns,consverbs = mywc.nouns.intersection(cons),mywc.verbs.intersection(cons)
                if len(antenouns) < 2 or len(consnouns) == 0:
                    continue;
                if len(antenouns) >= 1 and len(anteverbs) >= 1 and len(consnouns) >= 1 and len(consverbs) >= 1:
                    verbRules.add(row)
                if len(antenouns) >= 1 and len(consnouns) >= 1:
                    nounRules.add(row)
            #we should have short list of rules, let us weed out converse rules by lift or Conviction
            i=0
            midrules=verbRules.union(nounRules)
            #print(verbRules), print(nounRules)
            midlist=list(midrules)
            finalrules=set()
            antelist,conslist,liftlist,convlist=[],[],[],[]
            for rule in midlist:
                words = rule.split(',')
                antelist.append(words[1]), conslist.append(words[2]), liftlist.append(words[5]), convlist.append(words[6])
            for ante in antelist:
                i+=1
                try:
                    idx = conslist.index(ante)
                    if antelist[idx] == conslist[i-1]:
                        #mylist[i-1] and mylist[idx] are converse rules- Keep one with higher lift or conviction
                        lift1,conv1,lift2,conv2 = liftlist[i-1],convlist[i-1],liftlist[idx],convlist[idx]
                        if lift1 > lift2:
                            finalrules.add(midlist[i-1])
                        elif lift1 < lift2:
                            finalrules.add(midlist[idx])
                        elif conv1 > conv2:
                            finalrules.add(midlist[i-1])
                        else:
                            finalrules.add(midlist[idx])
                except ValueError:
                    finalrules.add(mylist[i-1])
                    continue
            for rule in finalrules:
                frout.write(str(rule) + '\n')

